In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.1 MB/s eta 0:00:00


In [ ]:
just_checking_integrity=False

In [ ]:
import gc
import pandas as pd
import scipy.sparse as sps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
valid=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [ ]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["UCF","ICBF","Graph","ICF"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/train/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


#del dataset_max
del df_feats
del df_max

0
loading UCF
18588456
loaded UCF
0
loading ICBF
16931962
loaded ICBF
0
loading Graph
9381571
loaded Graph
0
loading ICF
20341641
loaded ICF


In [ ]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/train/candidates.csv")
# if just_checking_integrity:
#     submission_df["score"]=submission_df["score"].apply(lambda x:x.replace("-inf","0")).apply(lambda x:x.replace("+inf","0")).apply(lambda x:x.replace("inf","0"))
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# else:
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [ ]:
# submission_df.head()

In [ ]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/train/candidates.csv")
# if just_checking_integrity:
#     submission_df_tr["score"]=submission_df_tr["score"].apply(lambda x:x.replace("-inf","0"))
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# else:
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [ ]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/train/candidates.csv")
if just_checking_integrity:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
else:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [ ]:
# dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
# dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")

dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")


In [ ]:
if just_checking_integrity:
    dataset=dataset[:int(0.5*len(dataset))]

In [ ]:
del dataset_max
# del submission_df
# del submission_df_tr

In [ ]:
import numpy as np
sessions=np.unique(dataset["Session_Id"])
train=pd.DataFrame()
train["Session_Id"]=sessions
if just_checking_integrity:
    train["Item_ID"]=bought[:len(sessions)]
else:
    train["Item_ID"]=bought
train["target"]=1

In [ ]:
gc.collect()

In [ ]:
dataset=dataset.merge(train,on=["Session_Id","Item_ID"],how="left")
dataset.fillna(0,inplace=True)
gc.collect()

In [ ]:
dataset["target"]=dataset["target"].astype("int8")
gc.collect()

In [ ]:
sessions_containing_bought_among_candidates=dataset[["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
sessions_containing_bought_among_candidates=sessions_containing_bought_among_candidates.loc[sessions_containing_bought_among_candidates["target"]>0]

dataset=dataset[dataset['Session_Id'].isin(sessions_containing_bought_among_candidates["Session_Id"].tolist())]

In [ ]:
NHITS=len(dataset['Session_Id'].unique())
HITRATE=NHITS/81600 #approximate value
HITRATE

In [ ]:
dataset.head(20)

In [ ]:
dataset

In [ ]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [ ]:
#dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

## K-Fold for All tradition models + LSTM models

In [ ]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [ ]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [ ]:
from sklearn.model_selection import KFold
from catboost import CatBoostRanker, Pool
import os
import time

start_time = time.time()

k = 10

score = 0
kf = KFold(n_splits=k)
index = 0

mrr = 0
mrr_session_count = 0

for train_index, valid_index in kf.split(train):

    train_sessions = train["Session_Id"][train_index].tolist()
    valid_sessions = train["Session_Id"][valid_index].tolist()

    train_set = dataset[dataset["Session_Id"].isin(train_sessions)]
    valid_set = dataset[dataset["Session_Id"].isin(valid_sessions)]

    train_set = train_set.sort_values("Session_Id")
    valid_set = valid_set.sort_values("Session_Id")

    qids_train = train_set["Session_Id"]
    qids_valid = valid_set["Session_Id"]

    X_train = train_set.drop(["target", "Session_Id", "Item_ID"], axis=1)
    y_train = train_set["target"]

    X_valid = valid_set.drop(["target", "Session_Id", "Item_ID"], axis=1)
    y_valid = valid_set["target"]

    train_pool = Pool(X_train, y_train, group_id=qids_train)
    valid_pool = Pool(X_valid, y_valid, group_id=qids_valid)

    model = CatBoostRanker(
        iterations=2000,
        loss_function='YetiRank',
        od_type='Iter',
        od_wait=20,
        depth=6,
        l2_leaf_reg=3,
        learning_rate=0.05,
        random_seed=42
    )
    ranker = model.fit(
        train_pool,
        eval_set=valid_pool,
        verbose=100,
        plot=False
    )
    ranker.save_model(f'./drive/MyDrive/Recommendation_system/rankers/catboost/all_traditional_and_lstm/model_{index}.cbm')

    # Get best score from evals_result
    best_iter = ranker.get_best_iteration()
    best_score = ranker.get_evals_result()
    score += best_score['validation']['PFound'][best_iter]

    # Calculate MRR@100
    predictions_rank = train_set
    pred = (model.predict(predictions_rank))
    predictions_rank["score"]=np.mean(pred ,axis=0)

    gc.collect()

    predictions_rank=predictions_rank[["Session_Id", "Item_ID", "target", "score"]]
    predictions_rank=predictions_rank.groupby(['Session_Id']).agg(list).reset_index()
    predictions_rank["indices"]=predictions_rank["score"].apply(lambda x: get_indices(x))
    predictions_rank["predictions"]=predictions_rank[["Item_ID","indices"]].apply(get_items,axis=1)
    predictions_rank=predictions_rank[["Session_Id","predictions"]]
    predictions_rank["rank"]=predictions_rank.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

    original_purchase_df=train_set[["Session_Id", "Item_ID", "target", "score"]]
    original_purchase_df=original_purchase_df[original_purchase_df['target'] == 1]
    original_purchase_df=original_purchase_df.drop(["target", "score"], axis=1)
    original_purchase_df.rename(columns={'Item_ID': 'purchase_item_id'}, inplace=True)

    merged_df = pd.merge(predictions_rank, original_purchase_df, on='Session_Id', how='inner')

    gc.collect()

    for _, row in merged_df.iterrows():
        predictions = row["predictions"]
        purchase_item_id = row["purchase_item_id"]
        indices = np.where(predictions == purchase_item_id)[0]
        first_index = indices.min() + 1 if len(indices) > 0 else -1

        mrr += 1 / first_index if first_index > 0 else 0

    mrr_session_count += merged_df.shape[0]

    index += 1

end_time = time.time()

elapsed_time = end_time - start_time


In [ ]:
mrr_final_performcance=(mrr / mrr_session_count) / k
print("overall MAP@100 = ", mrr_final_performcance)

In [ ]:
print("normalized score ", mrr_final_performcance*HITRATE)

In [ ]:
pairwise_final_performcance=score/k
print("overall Pairwise = ", pairwise_final_performcance)

In [ ]:
print("K-Fold execution time:", elapsed_time, "seconds")